# Validación en Cúmulos de Galaxias — EDR
Implementación inspirada en análisis tipo Bullet Cluster.
Se calculan:
- Perfil β del gas
- Masa visible (gas + galaxias)
- Masa adicional EDR
- Mapas de convergencia κ(x,y)
- Separación entre picos de masa visible y EDR

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import simps

## 1) Perfil β del gas del cúmulo
\begin{equation}
\rho_g(r) = \rho_0 \left(1 + (r/r_c)^2\right)^{-3\beta/2}
\end{equation}

In [ ]:
def rho_gas(r, rho0=1e14, rc=150, beta=0.7):
    return rho0 * (1 + (r/rc)**2)**(-1.5*beta)

def M_gas(r):
    rs = np.linspace(1e-3, r, 2000)
    integrand = 4*np.pi*rs**2 * rho_gas(rs)
    return simps(integrand, rs)

## 2) Masa EDR del cúmulo (flujo rotacional ampliado)

In [ ]:
def Omega_flow_cluster(r, Omega0=0.02, R_Omega=500):
    return Omega0 * np.exp(-r/R_Omega)

def rho_flow_cluster(r, kflow=0.02, eta=0.03, Omega0=0.02, R_Omega=500):
    return kflow * eta * Omega_flow_cluster(r, Omega0, R_Omega)**2

def M_EDR_cluster(r, kflow=0.02, eta=0.03, Omega0=0.02, R_Omega=500):
    rs = np.linspace(1e-3, r, 2000)
    integrand = 4*np.pi*rs**2 * rho_flow_cluster(rs, kflow, eta, Omega0, R_Omega)
    return simps(integrand, rs)

## 3) Mapa de masa superficial Σ(x,y)

In [ ]:
def Sigma_from_rho(rho_fn, x, y):
    R = np.sqrt(x**2 + y**2)
    z = np.linspace(-3000,3000,2000)
    integrand = rho_fn(np.sqrt(R**2 + z**2))
    return simps(integrand, z)

## 4) Generar mapas κ visibles y EDR

In [ ]:
grid = np.linspace(-1500,1500,200)
X, Y = np.meshgrid(grid, grid)

Sigma_vis = np.zeros_like(X)
Sigma_edr = np.zeros_like(X)

for i in range(len(grid)):
    for j in range(len(grid)):
        r = np.sqrt(X[i,j]**2 + Y[i,j]**2)
        Sigma_vis[i,j] = Sigma_from_rho(lambda rr: rho_gas(rr), X[i,j], Y[i,j])
        Sigma_edr[i,j] = Sigma_from_rho(lambda rr: rho_flow_cluster(rr), X[i,j], Y[i,j])

## 5) Visualización: contornos de masa visible y masa EDR

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.contour(X,Y,Sigma_vis,20)
plt.title('Masa visible')

plt.subplot(1,2,2)
plt.contour(X,Y,Sigma_edr,20)
plt.title('Masa EDR')
plt.show()